# 响应
可以在路径操作中使用 ```response_model``` 参数，值可以是个Pydantic 模型。用于限制返回的字段

```python
def func(response_model=Model):
    pass
```

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class userIn(BaseModel): # 这是请求模型，即用户需要提供的字段，包含密码
    username: str
    password: str
    email: str
    full_name: str | None = None

class userOut(BaseModel):  # 这是响应模型，即希望返回的字段，不希望返回明文密码
    username: str
    email: str
    full_name: str | None = None

@app.post("/user/", response_model=userOut)  # 通过 response_model 参数指定响应模型 
async def create_user(user: userIn):
    return user  # FastAPI 会自动将 userIn 转换为 userOut，并且过滤掉 password 字段

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

#### 一个应用的例子
在创建用户的场景里，我们需要：
1. 用户输入密码 ---- 定义 用户 模型
2. 但是在返回给用户的时候不希望返回密码 ---- 定义 返回给用户 模型
3. 返回给数据库时希望的是一个哈希加密的密码 ----- 定义 返回给数据库 模型

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr
import uvicorn

# 定义 用户 模型
class User(BaseModel):
    userName: str,
    email: EmailStr,
    password: str,

# 定义 返回给用户 模型
class ReponseUser(BaseModel):
    userName: str,
    email: EmailStr

# 定义 返回给数据库 模型
class toDB(BaseModel):
    userName: str,
    email: EmailStr,
    hashedPassword: str

# 定义生成hash加密 
def create_hash(password:int):
    hashedPassword = "hash"+password
    return hashedPassword

# 模拟在数据库创建数据
def create_in_DB(user: User):
    # 生成hash
    hashedPassword = create_hash(user.password)

    """
    构造符合toDB类型的内容，pydntic的.model_dump()方法将pydntic模型里的数据提取出来成为字典:
        这里提取出来的： dict{ "userName" : "用户名", "eamil":"邮件"}
    在字典前面加 ** 的方法将字典传给 一个函数 或者 类， 就会自动解析然后传入 值，而不是传入字典:
        **dict{ "userName" : "用户名", "eamil":"邮件"}，结果就是会提取出两个 值： userName="用户名", email="邮件"

    所以toDB(**ResponseUser.model_dump()) 等效于 toDB(userName="用户名", email="邮件"), 然后再加上hashedPassword
    所以 create_user就是一个toDB类型,并且有值：{"userName" : "用户名", "eamil":"邮件", "hashedPassword":"hashedPassword"}
    """
    create_user = toDB(**ResponseUser.model_dump(),hashedPassword=hashedPassword)
    
    return create_user

app = FastApi()
@app.post("/register")
async def create_user(user:User,response_model=ReponseUser):
    create_user = create_in_DB(user)
    return create_user  # 由于reponse_model的限制，只会返回 { "userName" : "用户名", "eamil":"邮件"}

#### response_model 中的 Union
response_model参数也可以是 Union 类型，比如 Union[ModelA, ModelB]。 这表示响应可以是 ModelA 或 ModelB 类型的实例。 FastAPI 会根据实际返回的数据类型来选择合适的模型进行验证和序列化。

In [ ]:
# 以上面的为例子
from typing import Union
app = FastApi()
@app.post("/register")
async def create_user(user:User,response_model=Union[toDB,ReponseUser]): # 返回可以是toDB或者ReponseUser类型的，取决于fastapi得到了哪种
    create_user = create_in_DB(user)
    return create_user 

#### response_model 的列表
response_model 参数还可以是一个包含 Pydantic 模型的列表类型，例如 response_model=List[Model]。 这表示响应将是一个模型实例的列表， FastAPI 会对列表中的每个元素进行验证和序列化。

基本上跟请求体是一个道理

In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str


items = [
    {"name": "Foo", "description": "There comes my hero"},
    {"name": "Red", "description": "It's my aeroplane"},
]


@app.get("/items/", response_model=list[Item])
async def read_items():
    return items

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
这里会返回
'[
    {"name":"Foo","description":"There comes my hero"},
    {"name":"Red","description":"It\'s my aeroplane"}
]'
"""

#### response_model 任意 dict的返回
如果希望返回任意的 dict 而不进行字段验证，可以将 response_model 设置为 dict 类型，例如 response_model=dict[str,int]。 这表示响应可以是任意的字典结构， 只要保证键和值的类型符合指定的类型提示即可。

In [ ]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/keyword-weights/", response_model=dict[str, float])
async def read_keyword_weights():
    return {"foo": 2.3, "bar": 3.4}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

# 响应状态码
可以在路径操作中使用 ```status_code``` 参数来声明用于响应的 HTTP 状态码：
```python
def func(status_code=201):
    pass
```

100 及以上状态码用于「消息」响应。你很少直接使用它们。具有这些状态代码的响应不能带有响应体。 200 及以上状态码用于「成功」响应。这些是你最常使用的。 200 是默认状态代码，它表示一切「正常」。 另一个例子会是 201，「已创建」。它通常在数据库中创建了一条新记录后使用。 一个特殊的例子是 204，「无内容」。此响应在没有内容返回给客户端时使用，因此该响应不能包含响应体。 300 及以上状态码用于「重定向」。具有这些状态码的响应可能有或者可能没有响应体，但 304「未修改」是个例外，该响应不得含有响应体。 400 及以上状态码用于「客户端错误」响应。这些可能是你第二常使用的类型。 一个例子是 404，用于「未找到」响应。 对于来自客户端的一般错误，你可以只使用 400。 500 及以上状态码用于服务器端错误。你几乎永远不会直接使用它们。当你的应用程序代码或服务器中的某些部分出现问题时，它将自动返回这些状态代码之一。

可以使用来自 fastapi.status 的便捷变量。

它们只是一种便捷方式，它们具有同样的数字代码，但是这样使用就可以使用编辑器的自动补全功能来查找它们。

In [ ]:
import uvicorn
from fastapi import FastAPI, status
app = FastAPI()

@app.post("/items/", status_code=status.HTTP_201_CREATED)  # 等效于 status_code=201
async def create_item(name: str):
    return {"name": name}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
得到response后，用 print(response.status_code) 查看 (输出201)
"""